### 0.Import some packages


In [ ]:
from keras.datasets import mnist
from PIL import Image
!pip install numpy==1.25.0
import numpy as np
np.random.seed(10) #for reproduceability


### 1. Define  Image correlation and convolution

In [ ]:
#this function is used for forward pass
def image_correlation(input, kernel, mode='valid'):
    # 'valid' ensures no padding (output size = input_size - kernel_size + 1).
    # 28 x 28 --> 26 x 26
    output_shape = [input.shape[0] - kernel.shape[0] + 1, input.shape[1] - kernel.shape[1] + 1]
    output = np.zeros(output_shape)

    #iterating over the input image
    for i in range(output.shape[0]):
        for j in range(output.shape[1]):
            #extract the 3 x 3 patch of the image at position i,j and aply kernel
            #as shown in fig 3
            output[i, j] = np.sum(input[i:i+kernel.shape[0], j:j+kernel.shape[1]] * kernel)
    return output

In [ ]:
# this function is used for back pass
def image_convolution(input, kernel, mode='full'):
    #this flips the kernel left/right followed by up/down
    kernel = np.flipud(np.fliplr(kernel))

    # padding ensures the output size is input_size + kernel_size - 1
    # retains the original size
    if mode == 'full':
    # Adds kernel_height - 1 pixels top/bottom.
    # Adds kernel_width - 1 pixels left/right.
        padded_input = np.pad(input, [(kernel.shape[0]-1, kernel.shape[0]-1), (kernel.shape[1]-1, kernel.shape[1]-1)], mode='constant')
    else:
        padded_input = input
    output_shape = [padded_input.shape[0] - kernel.shape[0] + 1, padded_input.shape[1] - kernel.shape[1] + 1]
    # initializing the output array.
    output = np.zeros(output_shape)

    for i in range(output.shape[0]):
        for j in range(output.shape[1]):
            # output os the  the product of overlapping elements between the padded input and kernel.
            output[i, j] = np.sum(padded_input[i:i+kernel.shape[0], j:j+kernel.shape[1]] * kernel)

    return output

In [ ]:
def image_convolution(input, kernel, mode='full'):
    #this flips the kernel for kernel operations
    kernel = np.flipud(np.fliplr(kernel))
    #padding the input image around the edges.
    if mode == 'full':
        padded_input = np.pad(input, [(kernel.shape[0]-1, kernel.shape[0]-1), (kernel.shape[1]-1, kernel.shape[1]-1)], mode='constant')
    else:
        padded_input = input

    output_shape = [padded_input.shape[0] - kernel.shape[0] + 1, padded_input.shape[1] - kernel.shape[1] + 1]
    # initializing the output array.
    output = np.zeros(output_shape)

    for i in range(output.shape[0]):
        for j in range(output.shape[1]):
            # output os the  the product of overlapping elements between the padded input and kernel.
            output[i, j] = np.sum(padded_input[i:i+kernel.shape[0], j:j+kernel.shape[1]] * kernel)

    return output

In [ ]:
class convolution:
    def __init__(self, input_shape, kernel_size, depth):
        input_depth, input_height, input_width = input_shape
        #defining the attributes for forward and backward methods
        self.depth = depth
        self.input_shape = input_shape
        self.input_depth = input_depth
        self.output_shape = (depth, input_height - kernel_size + 1, input_width - kernel_size + 1)
        self.kernels_shape = (depth, input_depth, kernel_size, kernel_size)

        #initialize randomly initially
        self.kernels = np.random.randn(*self.kernels_shape)
        self.biases = np.random.randn(*self.output_shape)


    def forward(self, input):
        self.input = input
        self.output = np.copy(self.biases)
        for i in range(self.depth):
            for j in range(self.input_depth):
                #input image correlation with correspoding kernels
                self.output[i] += image_correlation(self.input[j], self.kernels[i, j], 'valid')
        return self.output

    def backward(self, output_gradient, learning_rate):
        kernels_gradient = np.zeros(self.kernels_shape)
        input_gradient = np.zeros(self.input_shape)

        for i in range(self.depth):
            for j in range(self.input_depth):
                kernels_gradient[i, j] = image_correlation(self.input[j], output_gradient[i], "valid")
                #computing the gradients with respect to ip
                input_gradient[j] += image_convolution(output_gradient[i], self.kernels[i, j], "full")

        #updates
        self.kernels -= learning_rate * kernels_gradient
        self.biases -= learning_rate * output_gradient
        return input_gradient


In [ ]:
class sigmoid:
    def forward(self, x):
        self.output = 1 / (1 + np.exp(-x))
        return self.output

    def backward(self, output_gradient):
        # The derivative of the sigmoid function is sigmoid(x) * (1 - sigmoid(x))
        return output_gradient * self.output * (1 - self.output)

In [ ]:
class average_pooling:
    # Initializing the average pooling layer
    def __init__(self, input_shape=(2, 26, 26), pool_size=2, stride=2):
        # Storing the  parameters
        self.input_shape = input_shape
        self.pool_size = pool_size
        self.stride = stride

        # Calculating the output shape based on the input shape, pool size, and stride
        self.output_shape = (
            input_shape[0],
            (input_shape[1] - pool_size) // stride + 1,
            (input_shape[2] - pool_size) // stride + 1
        )

    # Implementing the forward pass for the pooling operation
    def forward(self, input):
        # Storing the input for potential future use
        self.input = input

        # Extracting the dimensions of the input
        depth, input_height, input_width = self.input_shape

        # Initializing an array for the output with the calculated output shape
        output = np.zeros(self.output_shape)

        # Iterating through each depth slice of the input
        for d in range(depth):
            # Iterating over the input's height and width with the specified stride
            for i in range(0, input_height, self.stride):
                for j in range(0, input_width, self.stride):
                    # Defining the current window fur pooling
                    h_start, h_end = i, i + self.pool_size
                    w_start, w_end = j, j + self.pool_size

                    # Extracting the current window from the ip
                    window = input[d, h_start:h_end, w_start:w_end]

                    # Calculating the average value of the current window
                    # and storing it in the corresponding position in the output
                    output[d, i // self.stride, j // self.stride] = np.mean(window)

        # Returning the op after pooling
        return output


    def backward(self, output_gradient):
        depth, input_height, input_width = self.input_shape
        input_gradient = np.zeros(self.input_shape)

        for d in range(depth):
            for i in range(0, input_height, self.stride):
                for j in range(0, input_width, self.stride):
                    h_start, h_end = i, i + self.pool_size
                    w_start, w_end = j, j + self.pool_size

                    # Distribute the gradient equally to each element in the window
                    input_gradient[d, h_start:h_end, w_start:w_end] += \
                        output_gradient[d, i // self.stride, j // self.stride] / (self.pool_size * self.pool_size)

        return input_gradient


In [ ]:
class flatten_layer:
    def forward(self, input):
        self.input_shape = input.shape
        return input.flatten()

    def backward(self, output_gradient):
        return output_gradient.reshape(self.input_shape)


In [ ]:
class dense_layer:
    def __init__(self, input_size, output_size):
        self.weights = np.random.randn(output_size, input_size)
        self.biases = np.random.randn(output_size)

    def forward(self, input):
        self.input = input
      #  print(self.input.shape)
        eval = np.dot(self.weights, input) + self.biases
      #  print(eval.shape) = 10,
        return eval

    def backward(self, output_gradient, learning_rate):
        # Gradient of loss w.r.t. weights
        weights_gradient = np.dot(output_gradient.reshape(-1, 1), self.input.reshape(1, -1))

        # Gradient of loss w.r.t. biases
        biases_gradient = output_gradient

        # Gradient computation of loss w.r.t. input
        input_gradient = np.dot(self.weights.T, output_gradient)

        # Updating weights and biases
        self.weights -= learning_rate * weights_gradient
        self.biases -= learning_rate * biases_gradient

        return input_gradient

In [ ]:
class softmax:
    def forward(self, input):
        exps = np.exp(input - np.max(input, axis=-1, keepdims=True))
        self.output = exps / np.sum(exps, axis=-1, keepdims=True)
        return self.output

# Softmax backward is a pass-through because:
    def backward(self, dL_dZ):
        return dL_dZ
        #just propagate p-t as downstream!

In [ ]:
#one hot code for all the labels
def one_hot_encode(y):
    one_hot = np.zeros((y.size, y.max()+1))
    one_hot[np.arange(y.size), y] = 1
    return one_hot

#Eg: y = np.array([0, 2, 1, 2]) becomes:
#[[1 0 0]
#[0 0 1]
#[0 1 0]
#[0 0 1]]

In [ ]:
class cross_entropy_loss:
    def __init__(self):
        pass

    def compute_loss(self, t_list, p_list):
        # Ensure inputs are 2D arrays
        t_list = np.atleast_2d(np.float_(t_list))
        p_list = np.atleast_2d(np.float_(p_list))

        # Compute cross entropy loss
        # we dont want p_list to be 0 so adding some error term
        losses = -np.sum(t_list * np.log(p_list + 1e-15), axis=1)
        return np.mean(losses)

    def compute_dloss(self, t_list, p_list):
        return p_list - t_list

### Load data and create labels

In [ ]:
# Load MNIST dataset and apply one hot encoding to the labels
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

#batch, num_imgs,w,h
train_images = train_images.reshape((-1, 1, 28, 28)) / 255.0
test_images = test_images.reshape((-1, 1, 28, 28)) / 255.0

#one hot encode train and test labels
train_labels = one_hot_encode(train_labels)
test_labels = one_hot_encode(test_labels)


### Instantiate Layers

In [ ]:
# Initialize layers
cnn_layer = convolution(input_shape=(1, 28, 28), kernel_size=3, depth=2)
sigmoid_layer = sigmoid()  # Sigmoid layer after convolution
pooling_layer = average_pooling(input_shape=(2, 26, 26), pool_size=2, stride=2)
flatten_layer = flatten_layer()
dense_layer = dense_layer(input_size=13*13*2, output_size=10)
softmax_layer = softmax()
cross_entropy_loss = cross_entropy_loss()

# Training parameters
num_epochs = 10
learning_rate = 0.01

# Store results over time
train_accuracies = []
train_losses = []

### Train

In [ ]:
# loop for epochs
for epoch in range(num_epochs):
    correct_train_predictions = 0
    total_loss = 0

    #loop over all images
    for i in range(len(train_images)):
        image = train_images[i]
        label = train_labels[i]

        # Forward pass
        cnn_output = cnn_layer.forward(image)
        sigmoid_output = sigmoid_layer.forward(cnn_output)
        pooling_output = pooling_layer.forward(sigmoid_output)
        flattened_output = flatten_layer.forward(pooling_output)
        dense_output = dense_layer.forward(flattened_output)
        predictions = softmax_layer.forward(dense_output)

        #compute loss
        loss = cross_entropy_loss.compute_loss(label, predictions)
        total_loss += loss

        # Check the prediction accuracy
        if np.argmax(predictions) == np.argmax(label):
            correct_train_predictions += 1

        # Backward pass
        grad_back = cross_entropy_loss.compute_dloss(label, predictions)
        #print(grad_back)
        grad_back = softmax_layer.backward(grad_back)
        #print(grad_back)
        grad_back = dense_layer.backward(grad_back, learning_rate)
        #print(grad_back)
        grad_back = flatten_layer.backward(grad_back)
        #print(grad_back)
        grad_back = pooling_layer.backward(grad_back)
        #print(grad_back)
        grad_back = sigmoid_layer.backward(grad_back)
        #print(grad_back)
        grad_back = cnn_layer.backward(grad_back, learning_rate)

    # Calculate average accuracy and loss for the epoch
    train_accuracy = correct_train_predictions / len(train_images)
    average_loss = total_loss / len(train_images)
    #print(average_losss)
    train_accuracies.append(train_accuracy)
    train_losses.append(average_loss)


    # Print summary after each epoch
    print(f"Epoch {epoch + 1}, Training Accuracy: {train_accuracy * 100:.2f}%, Average Loss: {average_loss:.4f}")



Epoch 1, Training Accuracy: 78.49%, Average Loss: 0.6870
Epoch 2, Training Accuracy: 88.71%, Average Loss: 0.3799
Epoch 3, Training Accuracy: 90.31%, Average Loss: 0.3289
Epoch 4, Training Accuracy: 91.53%, Average Loss: 0.2917
Epoch 5, Training Accuracy: 92.45%, Average Loss: 0.2603
Epoch 6, Training Accuracy: 93.21%, Average Loss: 0.2341
Epoch 7, Training Accuracy: 93.78%, Average Loss: 0.2128
Epoch 8, Training Accuracy: 94.24%, Average Loss: 0.1956
Epoch 9, Training Accuracy: 94.66%, Average Loss: 0.1818
Epoch 10, Training Accuracy: 95.03%, Average Loss: 0.1706


In [ ]:
def test_model(cnn_layer, sigmoid_layer, pooling_layer, flatten_layer, dense_layer, softmax_layer, test_images, test_labels):
    correct_test_predictions = 0
    total_loss_test = 0
    for i in range(len(test_images)):
        # Forward pass with test data
        test_image = test_images[i]
        test_label = test_labels[i]

        test_cnn_output = cnn_layer.forward(test_image)
        test_sigmoid_output = sigmoid_layer.forward(test_cnn_output)
        test_pooling_output = pooling_layer.forward(test_sigmoid_output)
        test_flattened_output = flatten_layer.forward(test_pooling_output)
        test_dense_output = dense_layer.forward(test_flattened_output)
        test_predictions = softmax_layer.forward(test_dense_output)

        loss_test = cross_entropy_loss.compute_loss(test_label, test_predictions)
        total_loss_test += loss_test

        # Check the prediction accuracy
        if np.argmax(test_predictions) == np.argmax(test_label):
            correct_test_predictions += 1

    test_accuracy = correct_test_predictions / len(test_images)
    loss_final = total_loss_test / len(test_images)
    return test_accuracy,loss_final


In [ ]:
test_accuracy,loss_final = test_model(cnn_layer, sigmoid_layer, pooling_layer, flatten_layer, dense_layer, softmax_layer, test_images, test_labels)
print(f"Testing Accuracy: {test_accuracy * 100:.2f}%, Testting Loss: {loss_final:.4f}")

Testing Accuracy: 95.24%, Testting Loss: 0.1700
